In [147]:
import pandas as pd
import numpy as np
import sys

The general shape of the OSC metadata_file file is as follows:
```
accel_reading1
accel_reading2
accel_reading3
accel_reading4
accel_reading5
accel_reading6
accel_reading7
accel_reading8
accel_reading9
accel_reading10
location_reading
```
The code below: 
- rolls up all the `accel_reading_*` values and calculates the `average(magnitude(accel_reading*_x,accel_reading*_y,accel_reading*_z))`
- Assigns the "bumpiness score" to the location_reading.

The resultant shape is:
# `location_reading<>rolledUp_accelMagnitude`

In [170]:
df = pd.read_csv("972237_16905_5a11d7b0f1f80.txt",header=None,skiprows=1,delimiter=";",names=["timestamp","longitude","latitude","elevation","horizontal_accuracy","GPSspeed","yaw","pitch","roll","accelerationX","accelerationY","accelerationZ","pressure","compass","videoIndex","tripFrameIndex","gravityX","gravityY","gravityZ","OBD2speed","vertical_accuracy"])
df['bump']=0
locs =  df.loc[df['longitude'].notnull()].index
bump=[]

for idx,i in enumerate(locs):
    bump_t=[]
    try:
        if idx == 0:
            next_val = locs[0]
            for idx2 in xrange(0,i):
                a_x = df['accelerationX'][idx2]
                a_y = df['accelerationY'][idx2]
                a_z = df['accelerationZ'][idx2]
                bump_val = np.sqrt(a_x**2 + a_y**2 + a_z**2)
                bump_t.append(bump_val)
            bump_avg = np.nansum(bump_t) / float(np.count_nonzero(~np.isnan(bump_t)))
            #print "Assining ",bump_avg," to row",i
            df.loc[i,'bump'] = bump_avg
        else:
            next_val = locs[idx + 1]
    except IndexError:
        print "end"
        break
    for idx2 in xrange(i,next_val):
        a_x = df['accelerationX'][idx2]
        a_y = df['accelerationY'][idx2]
        a_z = df['accelerationZ'][idx2]
        bump_val = np.sqrt(a_x**2 + a_y**2 + a_z**2)
        bump_t.append(bump_val)
    bump_avg = np.nansum(bump_t) / float(np.count_nonzero(~np.isnan(bump_t)))
    #print "Assining ",bump_avg," to row",i
    df.loc[i,'bump'] = bump_avg
    #print "Avg of all the accels for ",idx, bump_avg
df.iloc[locs].to_csv('972237-parsed.csv')

end


In [167]:
df.iloc[locs].head(10)

,timestamp,longitude,latitude,elevation,horizontal_accuracy,GPSspeed,yaw,pitch,roll,accelerationX,...,pressure,compass,videoIndex,tripFrameIndex,gravityX,gravityY,gravityZ,OBD2speed,vertical_accuracy,bump
23,1.51112e+09,-118.433372,34.031434,60.617363,10.0,26.780001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.320790
49,1.51112e+09,-118.433658,34.031393,59.538322,10.0,26.780001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.162240
78,1.51112e+09,-118.433937,34.031353,58.923149,10.0,26.459999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.312999
105,1.51112e+09,-118.434221,34.031318,58.935234,10.0,26.459999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.150256
134,1.51112e+09,-118.434509,34.031272,59.370476,10.0,26.290001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.151002
160,1.51112e+09,-118.434796,34.031234,60.016899,10.0,26.620001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.260506
191,1.51112e+09,-118.435067,34.031197,59.576775,10.0,26.430000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.109713
218,1.51112e+09,-118.435346,34.031150,58.605339,10.0,26.059999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.176535
245,1.51112e+09,-118.435600,34.031113,57.605705,10.0,25.610001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.233509
271,1.51112e+09,-118.435874,34.031073,57.214409,10.0,25.830000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,0.137723
